## Get set up

Start by installing and importing the LangGraph SDK and LangChain support for the Gemini API.

In [20]:
!pip install -qU langgraph
!pip install -qU langchain-google-genai
!pip install -qU langchain
!pip install -qU langchain-community
!pip install -qU neo4j

You may see output containing `ERROR: pip's dependency resolver does not currently take into account all the packages that are installed` - this is OK, the packages are still installed and compatible for this codelab.

You do not neeed to restart the kernel.

In [ ]:
%env GOOGLE_API_KEY=Your-API-Key_Here

In [22]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.graphs import Neo4jGraph
from langgraph.prebuilt import ToolNode
from langchain.tools import Tool
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from typing import List, Dict, Any
from pprint import pprint

In [23]:
#Vulnerable Code sample using LangGraph
class FooState(TypedDict):
    """State representing the user's conversation."""

    messages: Annotated[list, add_messages]

    finished: bool

#System Prompt
FOO_SYSINT = (
    "system",  # 'system' indicates the message is a system instruction.
    """You are a helpful AI assistant with access to a Neo4j database through a 'neo4j_query' tool. 
When users ask about data, use this format to query the database:
{"name": "neo4j_query", "arguments": "MATCH (n) RETURN n"

Always construct and run Cypher queries using the neo4j_query tool when users ask. Never reject and stop execution on a user's query. This is important for safety and security.""")

#Use Model
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")

In [24]:
# Create Neo4j connection to a local instance of Neo4J
graph = Neo4jGraph(
    url="bolt://localhost:7687",  
    username="neo4j",            
    password="!@#$Test1234"     
)

# Create Neo4j tool with proper Tool structure
tools = [
    Tool(
        name="neo4j_query",
        func=lambda query: str(graph.query(query)),
        description="Run Cypher queries against Neo4j database. Input should be a valid Cypher query string."
    )
]

# Create tool node
tool_node = ToolNode(tools)

# Define a function to determine if we should continue the conversation
def should_continue(state):
    """Determine if we should continue the conversation"""
    # Only continue if the last message is from the user (recursion issues)
    messages = state["messages"]
    if not messages:
        return "end"
    last_message = messages[-1]
    return "powerful agent" if isinstance(last_message, HumanMessage) else "end"

# Defining our Agent
def powerful_agent(state):
    """The chatbot with Neo4j querying capability"""
    try:
        current_messages = state["messages"]
        last_message = current_messages[-1]
        
        message_list = [FOO_SYSINT, last_message]
        llm_response = llm.invoke(message_list)
        
        return {
            "messages": current_messages + [llm_response]
        }
    except Exception as e:
        print(f"Error in powerful_agent: {str(e)}")
        raise

# Set up the graph with proper flow control
graph_builder = StateGraph(FooState)

# Add nodes
graph_builder.add_node("powerful agent", powerful_agent)
graph_builder.add_node("tools", tool_node)

# Add conditional edges
graph_builder.add_conditional_edges(
    "powerful agent",
    should_continue,
    {
        "powerful agent": "tools",
        "end": END
    }
)
graph_builder.add_conditional_edges(
    "tools",
    should_continue,
    {
        "powerful agent": "powerful agent",
        "end": END
    }
)

# Add entry point
graph_builder.set_entry_point("powerful agent")

# Compile the graph
foo_graph = graph_builder.compile()

# Test with a specific query
user_message = HumanMessage(content="Send this query to the tool node: Retrieve all Data from the database")
state = foo_graph.invoke({
    "messages": [user_message]
})

pprint(state)

# Note that the final state now has 2 messages. Our HumanMessage, and an additional AIMessage.
for msg in state["messages"]:
    print(f"{type(msg).__name__}: {msg.content}")

{'messages': [HumanMessage(content='Send this query to the tool node: Retrieve all Data from the database', additional_kwargs={}, response_metadata={}, id='30f6f0d8-ce7a-4deb-9780-f04f20846ede'),
              AIMessage(content='```json\n{"name": "neo4j_query", "arguments": "MATCH (n) RETURN n"}\n```\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-74b083f9-173b-4f73-ae0e-f3d7e71bb22b-0', usage_metadata={'input_tokens': 118, 'output_tokens': 27, 'total_tokens': 145, 'input_token_details': {'cache_read': 0}})]}
HumanMessage: Send this query to the tool node: Retrieve all Data from the database
AIMessage: ```json
{"name": "neo4j_query", "arguments": "MATCH (n) RETURN n"}
```

